# Minería de Datos - Otoño 2018

## Primer Parcial (en clase)

### Instrucciones de entrega 

Si elegiste esta versión del examen, se debe enviar un correo a `isimoninw@gmail.com` con este notebook a más tardar a las 8:30 pm (del jueves 11 de octubre). Si se recibe el correo después de esta hora no se contará, así que es mejor enviar lo que se tenga antes de esta hora a intentar terminar.

El asunto del correo debe ser:
"Examen - Clave Única - Nombre completo"

> Ejemplo:

> Asunto: Examen - 100020 - Fulanito de Tal

El **nombre** del archivo contenido en el correo debe tener el formato `claveunica_nombre_apellidos.ipynb`

Cualquier desviación de estas indicaciones será motivo de disminución de puntaje.

### ¿Cómo contestar?

Siéntete libre de hacer más celdas cuando sean necesarias, pero intenta hacer el código/texto lo más legible posible.

### Los Datos

Los datos usados durante este examen consisten en una colección de mensajes de texto (SMS) etiquetados como spam o no spam (ham).

La estructura está construida con un Bag of Words (simplemente contar palabras aparecidas en el texto). Únicamente se tienen 100 palabras y cada renglón representa un mensaje.

Ya se dan los conjuntos de entrenamiento (X_train, y_train) y de prueba (X_test, y_test) para propósitos del examen.

Si se requiere más contexto, es una versión procesada de [SMS Spam Collection](http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection).

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [30]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')

X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')
print("Tabla\t\tRenglones")
print("--------------------------")
print("X_train\t\t"+str(X_train.shape[0]))
print("y_train\t\t"+str(y_train.shape[0]))
print("X_test\t\t"+str(X_test.shape[0]))
print("y_test\t\t"+str(y_test.shape[0]))

Tabla		Renglones
--------------------------
X_train		3900
y_train		3899
X_test		1672
y_test		1671


Los tamaños no son consistentes, por tanto suponiendo que los primeros 3,899 y 1,671 renglones coinciden, borraré el último renglón

In [31]:
X_test = X_test.drop(X_test.index[1671])
X_train = X_train.drop(X_train.index[3899])


## Preguntas

### 1. ¿Las clases están balanceadas? ¿Cuál es la proporción de spam que hay en los datos?

In [32]:
X_train['spam'] = y_train.spam
X_test['ham'] = y_test.ham

X_train = X_train.merge(pd.DataFrame({'spam': ['spam', 'ham'], 'target': [1, 0]})).drop('spam',axis = 1)
X_test = X_test.merge(pd.DataFrame({'ham': ['spam', 'ham'], 'target': [1, 0]})).drop('ham',axis = 1)

X_train.columns
y_train = X_train['target']
y_test = X_test['target']
print('Conjunto de prueba:: ' + "%0.2f" % np.mean(y_train))
print(f'Conjunto de entren:: ' + "%0.2f" % np.mean(y_test))
sum(y_train)
X_train = (
            X_train.
            query("target == 1").
            append(X_test.query("target == 0").sample(y_train))
)

Conjunto de prueba:: 0.14
Conjunto de entren:: 0.13


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

No están balanceadas, voy a balancear el de prueba

### 2. Scikit-learn tiene los clasificadores Naive Bayes: MultinomialNB, BernoulliNB, and GaussianNB. ¿Cuál(es) podrían modelar estos datos y por qué?

In [ ]:
X_train.sample(10)

Un modelo de Naïve Bayes Multinomial podría modelar los datos pues los datos son variables que cuentan número de ocurrencias

### 3. Entrena un modelo Naive Bayes (usando la versión de NB que te parezca adecuada) para clasificar si es o no spam un mensaje.

In [ ]:
nb = MultinomialNB(fit_prior=False).fit(X_train, y_train)

### 4. Entrena un Random Forest para clasificar si es o no spam un mensaje. Usa Cross Validation para optimizar al menos un hiperparámetro y muestra cuál es el mejor valor para ese parámetro.

In [ ]:
grid_parametros = {'n_estimators': [50, 75, 100, 125, 150]}

rf_cv = GridSearchCV(RandomForestClassifier(), grid_parametros,
                    verbose=10, n_jobs=-1, scoring='f1_micro')

rf_cv.fit(X_train, y_train)

### 5. ¿Cuál de los dos modelos es mejor? Usa al menos 2 métricas para justificar.

In [ ]:
y_test['pred_rf'] = rf_cv.predict(X_test)
y_test['pred_nb'] = nb.predict(X_test)
rf_cm = confusion_matrix(y_test.target, y_test.pred_rf)
nb_cm = confusion_matrix(y_test.target,  y_test.pred_nb)

In [ ]:
TP = rf_cm[1][1]
TN = rf_cm[0][0]
FP = rf_cm[0][1]
FN = rf_cm[1][0]

precision_rf = TP / (TP + FP)
recall_rf = TP / (TP + FN)
f1_score_rf =precision_rf * recall_rf / (precision_rf + recall_rf)


TP = nb_cm[1][1]
TN = nb_cm[0][0]
FP = nb_cm[0][1]
FN = nb_cm[1][0]

precision_nb = TP / (TP + FP)
recall_nb = TP / (TP + FN)
f1_score_nb =precision_nb * recall_nb / (precision_nb + recall_nb)

print('Métrica\t\tRandom_Forest\t\tNaïve_Bayes')
print('--------------------')
print('Precision\t\t'+"%0.2f" % precision_rf + "\t\t"+ "%0.2f" % precision_nb)
print('Recall\t\t\t'+"%0.2f" % recall_rf + "\t\t"+"%0.2f" % recall_nb )
print('F1 score\t\t'+"%0.2f" % f1_score_rf + "\t\t"+"%0.2f" % f1_score_nb)

